In [5]:
import torch
from src.dataset import pca_data_loader
from src.model import DistillModel, StudentSequenceModel, config as cfg

In [6]:
# 加载模型
load_path = ".cache/student_model_2024-10-08-14-29-28.pth"
check_point = torch.load(load_path, weights_only=False)

radius_matrix = check_point["radius_matrix"]
model = DistillModel(radius_matrix, StudentSequenceModel)
model.student_model.load_state_dict(check_point["student_model"])

<All keys matched successfully>

In [7]:
# 加载测试数据
_, test_loader = pca_data_loader(n_components=10, batch_size=64)

Loading PCA data from F:\pyDistilledFDTD\dataset\.cache\pca\pca-components-10\batch-size-64.pkl


In [8]:
# 测试模型
model.eval()
model.set_simulation_mode(fdtd=True, lstm=True)
with torch.no_grad():
    for inputs, _ in test_loader:
        inputs = inputs.to(cfg.device)
        for data in inputs:
            fdtd_output, lstm_output = model(data.unsqueeze(0))
            
            fdtd_output = fdtd_output.squeeze(0)
            lstm_output = lstm_output.squeeze(0)
            
            relative_difference = torch.abs(fdtd_output - lstm_output) / torch.abs(fdtd_output)
            print(relative_difference)
            break
        break

tensor([5.4424, 0.3642, 1.0693, 0.2596, 0.0283, 2.5317, 4.1049, 3.4552, 0.6495,
        0.1731], device='cuda:0')
